In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Fits to Data With Spin-1 Unconstrained Waves and Both Reflectivities
This fit is very similar to our [previous study](./constrained.ipynb), but instead allows the D/S waves to be unconstrained to each other. So no D/S ratio or phase parameter is imposed here. Currently no bootstrap fits were done for these fits, but this may be updated in the future

Fit Details:
* Only spin-1 ($J=1$) waves
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D$}
* Include an isotropic background term
  * Assumed that sideband subtraction handles almost all background events, and so these events will be *true* $\omega$ events, but contribute in total a flat distribution to the angular distributions
* Uses all 4 polarization orientations
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 1.8$ 
* Perform 500 randomized fits in each bin of mass and $-t$ independently

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager
import neutralb1.analysis.statistics

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-1/unconstrained-ratio/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
t_bins = ["t_0.10-0.16", "t_0.16-0.23", "t_0.23-0.35", "t_0.35-1.00"]
binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        binned_results[t_bin] = ResultManager(**data)

binned_results: dict[str, ResultManager]

In [ ]:
for t_bin, results in binned_results.items():
    print(f"Results for t bin: {t_bin}")
    results.summary()

## Analysis

### Standard Plots
These plots are always handy to have

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(30, 6))
for i, (t_bin, results) in enumerate(binned_results.items()):    
    results.plot.intensity.jp(ax=axs[i])
    t_low, t_high = results.get_t_edges()    
    axs[i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV²")
    if i != 0:
        axs[i].set_ylabel("")
        axs[i].get_legend().remove()
plt.savefig(f"{STUDY_DIR}/jp_all_t_bins.pdf")    

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/waves.pdf")  
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves(fractional=True)
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/waves_fractional.pdf")  
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.diagnostic.matrix()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/matrix.pdf") 
    plt.show()

In [ ]:
sig_moments = set()
for t_bin, results in binned_results.items():    
    sig_moments = sig_moments.union(results.get_significant_moments(threshold=0.04))

fig, axs = plt.subplots(
    len(sig_moments), 4, figsize=(20, 25), layout="constrained", sharex=True, sharey="row"
)

row_max = {}
for col, (t_bin, results) in enumerate(binned_results.items()):    
    for row, moment in enumerate(sig_moments):        
        ax = axs[row, col]
        label = utils.convert_moment_name(moment) if col == 0 else ""
        my_kwargs = {moment: {"label": label}}
        results.plot.intensity.plot([moment], ax=ax, col_kwargs=my_kwargs)
        if col != 0:
            ax.set_ylabel("")
        max_val = ax.get_ylim()[1]
        print(moment)
        print(max_val)
        if row not in row_max or max_val > row_max[row]:
            row_max[row] = max_val
    t_low, t_high = results.get_t_edges()
    axs[0,col].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")


for col, (t_bin, results) in enumerate(binned_results.items()):
    for row, moment in enumerate(sig_moments):
        axs[row, col].set_ylim(top=row_max[row])
    

plt.savefig(f"{STUDY_DIR}/significant_moments_all_t_bins.pdf")


### D/S Ratio and Phase

In [ ]:
fig, axs = plt.subplots(
    3,
    4,    
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "m1pmS", "p1ppS", "m1ppS", "m1p0D"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()
plt.savefig(f"{STUDY_DIR}/ds_ratio_all_t_bins.pdf")

In [ ]:
fig, axs = plt.subplots(
    3,
    4,    
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "m1pmS", "m1ppS", "p1p0S", "m1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()

In [ ]:
fig, axs = plt.subplots(
    3,
    4,    
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves= ["m1pmS", "p1ppS", "m1ppS", "p1p0S", "m1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()

In [ ]:
fig, axs = plt.subplots(
    3,
    4,    
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "p1ppS", "p1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()

for ax in axs[1,:]:
    fig.delaxes(ax)
for ax in axs[2,:]:
    fig.delaxes(ax)

### $b_1(1235)$ and $1^{--}$ interference

In [ ]:
fig, axs = plt.subplots(
    2,
    4,
    sharex=True,        
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(30, 10),
    layout="constrained",
)
for i, (t_bin, results)in enumerate(binned_results.items()):
    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":colors[2], "alpha": 1.0},
        amp2_kwargs={"color":colors[3], "alpha": 1.0},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        axs[0,i].get_legend().remove()
    

max_y=0
for i in range(4):
    y_max = axs[0,i].get_ylim()[1]
    max_y = max(max_y, y_max)
for i in range(4):
    axs[0,i].set_ylim(0, max_y)


plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")

### $1^-P_0^{(\varepsilon)}$, $1^-P_{+1}^{(\varepsilon)}$ Waves
There's a possibility for the m=0 wave to have a $\rho(1700)$ contribution, while m=1 likely is $\rho(1450)$

In [ ]:
fig, axs = plt.subplots(
    1, 4, figsize=(20, 5), layout="constrained", sharex=True, sharey="row"
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    ax = axs[col]
    my_kwargs = {
        "p1m0P": {"color": "red"},
        "m1m0P": {"color": "blue"},
    }
    results.plot.intensity.plot(["p1m0P", "m1m0P"], ax=ax, col_kwargs=my_kwargs)
    t_low, t_high = results.get_t_edges()
    ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")
    if col != 0:
        ax.set_ylabel("")
        ax.get_legend().remove()

plt.savefig(f"{STUDY_DIR}/1m0P_all_t_bins.pdf")

In [ ]:
fig, axs = plt.subplots(
    1, 4, figsize=(20, 5), layout="constrained", sharex=True, sharey="row"
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    ax = axs[col]
    my_kwargs = {
        "p1mpP": {"color": "red"},
        "m1mpP": {"color": "blue"},
    }
    results.plot.intensity.plot(["p1mpP", "m1mpP"], ax=ax, col_kwargs=my_kwargs)
    t_low, t_high = results.get_t_edges()
    ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")
    if col != 0:
        ax.set_ylabel("")
        ax.get_legend().remove()


plt.savefig(f"{STUDY_DIR}/1mpP_all_t_bins.pdf")

### Naturalities

In [ ]:
fig, axs = plt.subplots(
    4, 4, figsize=(20, 15), layout="constrained", sharex=True, sharey="row"
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_values = ["p1p", "p1m", "m1p", "m1m"]
    labels = ["Natural $1^+$", "Natural $1^-$", "Unnatural $1^+$", "Unnatural $1^-$"]
    colors = ["tab:red", "tab:red", "tab:blue", "tab:blue"]
    for row in range(4):
        ax = axs[row, col]    
        label = labels[row] if col == 0 else ""
        val = row_values[row]
        my_kwargs = {val: {"label": label, "color": colors[row]}}
        results.plot.intensity.plot([val], ax=ax, col_kwargs=my_kwargs)

plt.savefig(f"{STUDY_DIR}/naturalities_all_t_bins.pdf")

### Simple Breit-Wigner Line Shape Fits to b1

#### S-wave

In [ ]:
from scipy.optimize import curve_fit
from neutralb1.analysis.physics import breit_wigner

vec_bw = np.vectorize(breit_wigner)
function_masses = np.arange(1.0, 1.5, 0.001)

def bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=0)    
    return scale * np.square(np.abs(bw))

fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey=True, sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = ["$1^+S_{1}^{(+)}$", "$1^+S_{0}^{(+)}$", "$1^+S_{-1}^{(+)}$"]
    row_wave = ["p1ppS", "p1p0S", "p1pmS"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        popt, pcov = curve_fit(
            bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=20)

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {mass_opt:.3f} ± {perr[1]:.3f} GeV\n"
            rf"$\Gamma$: {width_opt:.3f} ± {perr[2]:.3f} GeV"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=15,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_refl+_all_t_bins.pdf")


In [ ]:
from scipy.optimize import curve_fit
from neutralb1.analysis.physics import breit_wigner

vec_bw = np.vectorize(breit_wigner)
function_masses = np.arange(1.0, 1.5, 0.001)

def bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=0)    
    return scale * np.square(np.abs(bw))

fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey=True, sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = ["$1^+S_{1}^{(-)}$", "$1^+S_{0}^{(-)}$", "$1^+S_{-1}^{(-)}$"]
    row_wave = ["m1ppS", "m1p0S", "m1pmS"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        # we need to introduce bounds for negative reflecitivity waves to avoid unphysical fit results
        lower_bounds = [-np.inf, 1.1, 0.05]
        upper_bounds = [np.inf, 1.4, 0.4]

        popt, pcov = curve_fit(
            bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
            bounds=(lower_bounds, upper_bounds)
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=20)

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {mass_opt:.3f} ± {perr[1]:.3f} GeV\n"
            rf"$\Gamma$: {width_opt:.3f} ± {perr[2]:.3f} GeV"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=15,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_refl-_all_t_bins.pdf")

#### D-wave

In [ ]:
vec_bw = np.vectorize(breit_wigner)
function_masses = np.arange(1.0, 1.5, 0.001)

def bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=2) # L = D-wave
    return scale * np.square(np.abs(bw))

fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey=True, sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = ["$1^+D_{1}^{(+)}$", "$1^+D_{0}^{(+)}$", "$1^+D_{-1}^{(+)}$"]
    row_wave = ["p1ppD", "p1p0D", "p1pmD"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        popt, pcov = curve_fit(
            bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=20)

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {mass_opt:.3f} ± {perr[1]:.3f} GeV\n"
            rf"$\Gamma$: {width_opt:.3f} ± {perr[2]:.3f} GeV"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=15,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1D_refl+_all_t_bins.pdf")


In [ ]:
vec_bw = np.vectorize(breit_wigner)
function_masses = np.arange(1.0, 1.5, 0.001)

def bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=2) # L = D-wave
    return scale * np.square(np.abs(bw))

fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey=True, sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = ["$1^+D_{1}^{(-)}$", "$1^+D_{0}^{(-)}$", "$1^+D_{-1}^{(-)}$"]
    row_wave = ["m1ppD", "m1p0D", "m1pmD"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       
        
        # we need to introduce bounds for negative reflecitivity waves to avoid unphysical fit results
        lower_bounds = [-np.inf, 1.1, 0.05]
        upper_bounds = [np.inf, 1.4, 0.4]

        popt, pcov = curve_fit(
            bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
            bounds=(lower_bounds, upper_bounds)
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=20)

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {mass_opt:.3f} ± {perr[1]:.3f} GeV\n"
            rf"$\Gamma$: {width_opt:.3f} ± {perr[2]:.3f} GeV"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=15,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1D_refl-_all_t_bins.pdf")


## Constrained Vs Unconstrained D/S Ratio

In [ ]:
old_binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}../constrained-ratio/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        old_binned_results[t_bin] = ResultManager(**data)
old_binned_results: dict[str, ResultManager]

### S and D-wave Comparisons

#### Natural

In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["p1ppS", "p1p0S", "p1pmS"]
    row_ylabel = ["$1^+S_{1}^{(+)}$", "$1^+S_{0}^{(+)}$", "$1^+S_{-1}^{(+)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = old_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = old_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            color="tab:red",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:red",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20)
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["p1ppD", "p1p0D", "p1pmD"]
    row_ylabel = ["$1^+D_{1}^{(+)}$", "$1^+D_{0}^{(+)}$", "$1^+D_{-1}^{(+)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = old_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = old_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            color="tab:red",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:red",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20)
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


#### Unnatural

In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["m1ppS", "m1p0S", "m1pmS"]
    row_ylabel = ["$1^+S_{1}^{(-)}$", "$1^+S_{0}^{(-)}$", "$1^+S_{-1}^{(-)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = old_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = old_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            color="tab:blue",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:blue",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20)
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["m1ppD", "m1p0D", "m1pmD"]
    row_ylabel = ["$1^+D_{1}^{(-)}$", "$1^+D_{0}^{(-)}$", "$1^+D_{-1}^{(-)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = old_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = old_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            color="tab:blue",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:blue",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20)
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


### Likelihood Comparison

In [ ]:
# [2 (real, imag) params per amp] * [2 (reflectivities)] * [3 (spin projections)] * [3 (angular momenta: S, P, D)] - [2 reference phases]
unconstrained_free_params = 2 * 2 * 3 * 3 - 2

# [2 (real, imag) params per amp] * [2 (reflectivities)] * [3 (spin projections)] * [2 (angular momenta: S, P)] - [2 reference phases] + [2 (mag, phase) params for D-wave]
# the D/S constraint effectively removes the D-wave amplitude parameters from the free parameters, replacing them with 2 new parameters (magnitude and phase of D/S ratio)
constrained_free_params = 2 * 2 * 3 * 2 - 2 + 2

fig, axs = plt.subplots(
    1, 4, figsize=(20, 5), layout="constrained", sharex=True,
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    unconstrained_likelihood = results.fit_df["likelihood"]
    constrained_likelihood = old_binned_results[t_bin].fit_df["likelihood"]

    delta_likelihood = unconstrained_likelihood - constrained_likelihood
    delta_dof = unconstrained_free_params - constrained_free_params

    ax = axs[col]
    # histogram the LRT (delta_likelihood / delta_dof)
    ax.hist(delta_likelihood / delta_dof, bins=30, alpha=0.7, color="tab:blue", edgecolor="black")
    t_low, t_high = results.get_t_edges()
    ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)
    ax.set_xlabel("Likelihood Ratio Test Statistic (per dof)", fontsize=15)
    ax.set_ylabel("Counts", fontsize=15)



## Cleanup
Create zips of all the plots

In [ ]:
%%bash
cd ${STUDY_DIR}
zip all_t_bin_plots.zip *.pdf
zip -r t_bin_plots.zip t_*/plots/*.pdf